# 第二章：处理文本数据

本notebook中使用的包：

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.5.1
tiktoken version: 0.7.0


- 本章涵盖数据准备和采样，以使输入数据为大语言模型"准备就绪"

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/01.webp?timestamp=1" width="500px">

## 2.1 理解词嵌入

- 本节没有代码

- 嵌入有多种形式；我们在本书中专注于文本嵌入

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/02.webp" width="500px">

- 大语言模型在高维空间中处理嵌入（即数千个维度）
- 由于我们无法可视化如此高维的空间（我们人类只能思考1、2或3维），下图展示了一个2维嵌入空间

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/03.webp" width="300px">

## 2.2 文本分词

- 在本节中，我们对文本进行分词，这意味着将文本分解为更小的单位，如单个单词和标点符号

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/04.webp" width="300px">

- 加载我们想要处理的原始文本
- [伊迪丝·华顿的《判决》](https://en.wikisource.org/wiki/The_Verdict)是一个公共领域的短篇小说

In [2]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

<br>

---

<br>

#### 解决SSL证书错误

- 一些读者报告在VSCode或Jupyter中运行`urllib.request.urlretrieve`时看到ssl.SSLCertVerificationError: `SSL: CERTIFICATE_VERIFY_FAILED`错误。
- 这通常意味着Python的证书包已过时。


**解决方案**

- 使用Python ≥ 3.9；您可以通过执行以下代码检查Python版本：
```python
import sys
print(sys.__version__)
```
- 升级证书包：
  - pip: `pip install --upgrade certifi`
  - uv: `uv pip install --upgrade certifi`
- 升级后重启Jupyter内核。
- 如果您在执行前面的代码单元格时仍然遇到`ssl.SSLCertVerificationError`错误，请查看[GitHub上的更多信息讨论](https://github.com/rasbt/LLMs-from-scratch/pull/403)

<br>

---

<br>

In [3]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
    
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- 目标是对此文本进行分词和嵌入，供大语言模型使用
- 让我们基于一些简单的示例文本开发一个简单的分词器，然后可以将其应用到上面的文本
- 以下正则表达式将在空白处分割

In [4]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 我们不仅想在空白处分割，还想在逗号和句号处分割，所以让我们修改正则表达式来实现这一点

In [5]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 正如我们所看到的，这会产生空字符串，让我们删除它们

In [6]:
# 去除每个项目的空白，然后过滤掉任何空字符串。
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 这看起来很好，但让我们也处理其他类型的标点符号，如句号、问号等

In [7]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 这很好，我们现在准备将此分词应用到原始文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/05.webp" width="350px">

In [8]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- Let's calculate the total number of tokens

In [9]:
print(len(preprocessed))

4690


## 2.3 将token转换为token ID

- 接下来，我们将文本token转换为token ID，稍后可以通过嵌入层进行处理

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/06.webp" width="500px">

- 从这些token中，我们现在可以构建一个包含所有唯一token的词汇表

In [ ]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [ ]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 以下是此词汇表中的前50个条目：

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 下面，我们使用一个小词汇表来说明短示例文本的分词：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/07.webp?123" width="500px">

- 现在将所有内容组合到一个分词器类中

In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 在指定标点符号之前替换空格
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode`函数将文本转换为token ID
- `decode`函数将token ID转换回文本

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/08.webp?123" width="500px">

- 我们可以使用分词器将文本编码（即分词）为整数
- 这些整数随后可以（稍后）作为大语言模型的输入进行嵌入

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 我们可以将整数解码回文本

In [ ]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [ ]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 添加特殊上下文token

- 为未知单词添加一些特殊token并表示文本的结尾是很有用的

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/09.webp?123" width="500px">

- 一些分词器使用特殊token来帮助大语言模型获取额外的上下文
- 这些特殊token包括
  - `[BOS]`（序列开始）标记文本的开始
  - `[EOS]`（序列结束）标记文本结束的位置（这通常用于连接多个不相关的文本，例如两篇不同的维基百科文章或两本不同的书等）
  - `[PAD]`（填充）如果我们以大于1的批次大小训练大语言模型（我们可能包含多个不同长度的文本；使用填充token，我们将较短的文本填充到最长长度，使所有文本具有相等的长度）
- `[UNK]`用于表示词汇表中未包含的单词

- 请注意，GPT-2不需要上述任何这些token，只使用`<|endoftext|>` token来降低复杂性
- `<|endoftext|>`类似于上面提到的`[EOS]` token
- GPT还使用`<|endoftext|>`进行填充（因为我们通常在批量输入训练时使用掩码，无论如何我们都不会关注填充的token，所以这些token是什么并不重要）
- GPT-2不使用`<UNK>` token来处理词汇表外的单词；相反，GPT-2使用字节对编码（BPE）分词器，它将单词分解为子词单元，我们将在后面的章节中讨论



- 我们在两个独立的文本源之间使用`<|endoftext|>` token：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/10.webp" width="500px">

- 让我们看看如果对以下文本进行分词会发生什么：

In [17]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 上面的代码产生了一个错误，因为单词"Hello"不在词汇表中
- 为了处理这种情况，我们可以向词汇表添加特殊token，如`"<|unk|>"的token，它在GPT-2训练中用于表示文本的结尾（它也用于连接的文本之间，比如我们的训练数据集由多篇文章、书籍等组成）
- Since we are already extending the vocabulary, let's add another token called `"<|endoftext|>"` which is used in GPT-2 training to denote the end of a text (and it's also used between concatenated text, like if our training datasets consists of multiple articles, books, etc.)

In [18]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [19]:
len(vocab.items())

1132

In [20]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 我们还需要相应地调整分词器，使其知道何时以及如何使用新的`<unk>` token

In [21]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 在指定标点符号之前替换空格
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

让我们尝试使用修改后的分词器对文本进行分词：

In [22]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [23]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [24]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 字节对编码

- GPT-2使用字节对编码（BPE）作为其分词器
- 它允许模型将不在其预定义词汇表中的单词分解为更小的子词单元或甚至单个字符，使其能够处理词汇表外的单词
- 例如，如果GPT-2的词汇表中没有单词"unfamiliarword"，它可能会将其分词为["unfam", "iliar", "word"]或其他某种子词分解，这取决于其训练的BPE合并
- 原始的BPE分词器可以在这里找到： [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 在本章中，我们使用OpenAI开源[tiktoken](https://github.com/openai/tiktoken)库中的BPE分词器，它在Rust中实现了核心算法以提高计算性能
- 我在[./bytepair_encoder](../02_bonus_bytepair-encoder)中创建了一个notebook，并排比较这两个实现（在示例文本上tiktoken快了约5倍）

In [25]:
# pip install tiktoken

In [26]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [27]:
tokenizer = tiktoken.get_encoding("gpt2")

In [28]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [29]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE分词器将未知单词分解为子词和单个字符：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/11.webp" width="300px">

## 2.6 使用滑动窗口进行数据采样

- 我们训练大语言模型一次生成一个单词，因此我们希望相应地准备训练数据，其中序列中的下一个单词表示要预测的目标：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/12.webp" width="400px">

In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 对于每个文本块，我们需要输入和目标
- 由于我们希望模型预测下一个单词，因此目标是向右移动一个位置的输入

In [31]:
enc_sample = enc_text[50:]

In [32]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 逐个来看，预测将如下所示：

In [33]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 我们将在涵盖注意力机制后的后续章节中处理下一个单词的预测
- 现在，我们实现一个简单的数据加载器，它遍历输入数据集并返回移位一个位置的输入和目标

- 安装并导入PyTorch（有关安装提示，请参见附录A）

In [35]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.5.1


- 我们使用滑动窗口方法，将位置更改+1：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">

- 创建从输入文本数据集中提取块的数据集和数据加载器

In [36]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # 对整个文本进行分词
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # 使用滑动窗口将文本分块为max_length的重叠序列
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [37]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # 初始化分词器
    tokenizer = tiktoken.get_encoding("gpt2")

    # 创建数据集
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 创建数据加载器
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

- 让我们使用批次大小为1、上下文大小为4的大语言模型测试数据加载器：

In [38]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [40]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 如下所示，使用等于上下文长度（此处为4）的步长的示例：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/14.webp" width="500px">

- 我们还可以创建批处理输出
- 请注意，我们在这里增加步长，以便批次之间没有重叠，因为更多的重叠可能导致过拟合增加

In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 创建token嵌入

- 数据已经几乎准备好用于大语言模型
- 但最后让我们使用嵌入层将token嵌入到连续的向量表示中
- 通常，这些嵌入层是大语言模型本身的一部分，并在模型训练期间更新（训练）

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/15.webp" width="400px">

- 假设我们有以下四个输入示例，其输入ID为2、3、5和1（分词后）：

In [42]:
input_ids = torch.tensor([2, 3, 5, 1])

- 为简单起见，假设我们有一个只有6个单词的小词汇表，我们想创建大小为3的嵌入：

In [43]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 这将产生一个6x3的权重矩阵：

In [44]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 对于熟悉独热编码的人来说，上面的嵌入层方法本质上只是实现独热编码后跟全连接层中矩阵乘法的更有效方式，这在[./embedding_vs_matmul](../03_bonus_embedding-vs-matmul)的补充代码中有描述
- 因为嵌入层只是一个更有效的实现，等同于独热编码和矩阵乘法方法，它可以被视为一个可以通过反向传播优化的神经网络层

- 要将ID为3的token转换为3维向量，我们执行以下操作：

In [45]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 请注意，上面是`embedding_layer`权重矩阵中的第4行
- 要嵌入上面所有四个`input_ids`值，我们执行

In [46]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 嵌入层本质上是一个查找操作：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/16.webp?123" width="500px">

- **您可能对比较嵌入层与常规线性层的额外内容感兴趣：[../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)**

## 2.8 编码词位置

- 嵌入层将ID转换为相同的向量表示，无论它们在输入序列中的位置如何：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/17.webp" width="400px">

- 位置嵌入与token嵌入向量结合，形成大语言模型的输入嵌入：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/18.webp" width="500px">

- 字节对编码器的词汇表大小为50,257：
- 假设我们想将输入token编码为256维向量表示：

In [47]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 如果我们从数据加载器中采样数据，我们将每个批次中的token嵌入到256维向量中
- 如果我们的批次大小为8，每个批次有4个token，这将产生一个8 x 4 x 256的张量：

In [48]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [49]:
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [50]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

# 取消注释并执行以下行以查看嵌入的外观
# print(token_embeddings)

torch.Size([8, 4, 256])


- GPT-2使用绝对位置嵌入，因此我们只需创建另一个嵌入层：

In [51]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# 取消注释并执行以下行以查看嵌入层权重的外观
# print(pos_embedding_layer.weight)

In [52]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

# 取消注释并执行以下行以查看嵌入的外观
# print(pos_embeddings)

torch.Size([4, 256])


- 要创建大语言模型中使用的输入嵌入，我们只需将token嵌入和位置嵌入相加：

In [53]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

# 取消注释并执行以下行以查看嵌入的外观
# print(input_embeddings)

torch.Size([8, 4, 256])


- 在输入处理工作流程的初始阶段，输入文本被分割成单独的token
- 在此分割之后，这些token根据预定义的词汇表转换为token ID：

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/19.webp" width="400px">

# 总结和要点

请参见[./dataloader.ipynb](./dataloader.ipynb)代码notebook，这是我们在本章中实现的数据加载器的简洁版本，我们将在后续章节中训练GPT模型时需要使用。

有关练习解决方案，请参见[./exercise-solutions.ipynb](./exercise-solutions.ipynb)。

如果您有兴趣了解如何从头开始实现和训练GPT-2分词器，请参见[字节对编码（BPE）分词器从头开始](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) notebook。